In [ ]:
import os
import torch
from unsloth import FastLanguageModel
from huggingface_hub import HfApi

def upload_checkpoint(checkpoint_path : str, repo_id : str):
    max_seq_length : int = 8192
    dtype = None
    load_in_4bit : bool = True

    try:
        model, tokenizer = FastLanguageModel.from_pretrained(
            checkpoint_path,
            max_seq_length=max_seq_length,
            dtype=dtype,
            load_in_4bit=load_in_4bit,
            device_map='auto'
        )

        branch_name : str = os.path.basename(checkpoint_path)
        
        # Create a new branch for this checkpoint
        api = HfApi()
        api.create_branch(repo_id=repo_id, branch=branch_name)

        # Push model and tokenizer to the new branch
        model.push_to_hub(repo_id=repo_id, revision=branch_name)
        tokenizer.push_to_hub(repo_id=repo_id, revision=branch_name)
        print(f"Uploaded checkpoint {checkpoint_path} to branch {branch_name}")
    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        # Delete model and tokenizer to free up memory
        del model
        del tokenizer

        # Clear CUDA cache
        if torch.cuda.is_available():
            torch.cuda.empty_cache()

In [ ]:
checkpoints_folder : str = "finetuning-scripts/finetune-gemma-2-2b-old/outputsgemma2-2b-it-esg"  # Replace with your folder path
repo_id : str = "chris7374/esgemma-2b-synthetic-data-final-2e-4"  # Hugging Face repository ID

#Iterate all subfolders and upload the model and tokenizer
for checkpoint in os.listdir(checkpoints_folder):
    checkpoint_path : str = os.path.join(checkpoints_folder, checkpoint)
    if os.path.isdir(checkpoint_path):
        upload_checkpoint(checkpoint_path=checkpoint_path, repo_id=repo_id)